In [1]:
!python -m pip install --upgrade pip
!pip install scikit-learn pandas cupy-cuda11x
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 38.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 MB 40.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 100.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 45.9 MB/s eta 0:00:00
Looking in in

In [19]:
dictionary = {
  34: 0, 7: 1, 2: 2, 8: 3,
        4: 4, 22: 5, 0: 6, 6: 7
}

import json
import numpy as np
import pickle
import pandas as pd

# Assuming x contains your features and y contains your classes


def parse_json(json_data):
    X = []
    y = []

    for key, value in json_data.items():
        encoded_array = value['x']
        decoded_array = pickle.loads(encoded_array.encode('latin1'))
        X.append(decoded_array)
        y.append(value['y'])

    X = np.array(X)
    y = np.array(y)

    return X, y

# # Example usage:
with open('../../data/output-balanced-classes-10000-or-more.json', 'r') as f:
    json_data = json.load(f)

x, y = parse_json(json_data)
x = np.reshape(x, (x.shape[0], -1))  # Reshape X to (10, 64)

# Combine x and y into a single DataFrame
df = pd.DataFrame(x)
df['y'] = y

# Calculate the counts of each class (y)
class_counts = df['y'].value_counts()

# Filter out classes with fewer than 10,000 occurrences
valid_classes = class_counts[class_counts >= 10000].index

# Remove rows where y has fewer than 10,000 occurrences
df_filtered = df[df['y'].isin(valid_classes)]
df_filtered.dropna(inplace=True)

# Separate x and y again
x = df_filtered.drop(columns=['y']).values
y = df_filtered['y'].values
y = np.vectorize(dictionary.get)(y)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 40: ordinal not in range(128)

In [17]:
# Install cuML if you haven't already
# !pip install cuml

import cupy as cp
from cuml.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib, pickle

# Assuming X contains your 8x8 matrices and y contains the corresponding classes
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# # # Normalizing the data
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert numpy arrays to cupy arrays for GPU acceleration
X_train_gpu = cp.array(X_train).astype(np.float32)
X_test_gpu = cp.array(X_test)
y_train_gpu = cp.array(y_train).astype(np.float32)
y_test_gpu = cp.array(y_test)

# Initializing and training the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_streams=1)
clf.fit(X_train_gpu, y_train_gpu)

# Evaluate the model
train_accuracy = clf.score(X_train_gpu, y_train_gpu)
test_accuracy = clf.score(X_test_gpu, y_test_gpu)
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Obtain predictions on the test data
rfc_test_predictions = clf.predict(X_test_gpu)

# Decode the predicted labels back to characters
# rfc_test_predictions = label_encoder.inverse_transform(cp.asnumpy(rfc_test_predictions).astype(int))

# Print or use the test predictions
print("Test Predictions:", rfc_test_predictions)

# Save the trained model for later use
joblib.dump(clf, 'random_forest_model.joblib')

# with open('standard_scaler.pkl', 'wb') as file:
#     pickle.dump(scaler, file)

Training Accuracy: 0.87379390001297
Testing Accuracy: 0.815599262714386
Test Predictions: [3. 7. 4. ... 2. 1. 2.]


['random_forest_model.joblib']

In [8]:
np.unique(y)

array([ 0,  2,  4,  6,  7,  8, 22, 34])

In [7]:
predictions.unique()

0    2
1    8
2    0
3    6
4    7
5    4
dtype: int64

In [13]:
y.shape

(420158,)

(29745,)

In [4]:
np.unique(rfc_test_predictions, return_counts=True)

(array([0., 1., 2., 3., 4., 5., 6., 7.]),
 array([2214, 2874, 8131, 5285, 2729, 1735, 3980, 2797]))

In [18]:
import cupy as cp
from cuml.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib, pickle

# Assuming X contains your 8x8 matrices and y contains the corresponding classes

# Splitting data into training and testing sets
print("Splitting data into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print("Data split completed.")

# Convert numpy arrays to cupy arrays for GPU acceleration
print("Converting numpy arrays to cupy arrays for GPU acceleration...")
X_train_gpu = cp.array(X_train).astype(np.float32)
X_test_gpu = cp.array(X_test).astype(np.float32)
y_train_gpu = cp.array(y_train)
y_test_gpu = cp.array(y_test)
print("Conversion to cupy arrays completed.")

# Explicitly convert cupy arrays to numpy arrays for compatibility with scikit-learn
print("Converting cupy arrays back to numpy arrays for compatibility with scikit-learn...")
X_train_np = X_train_gpu.get()
X_test_np = X_test_gpu.get()
y_train_np = y_train_gpu.get()
y_test_np = y_test_gpu.get()
print("Conversion to numpy arrays completed.")

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, ],
    'max_features': ['auto', 'sqrt', 'log2']
}
print("Parameter grid defined for grid search.")

# Initializing the Random Forest classifier
print("Initializing the Random Forest classifier...")
rf_clf = RandomForestClassifier(random_state=42, n_streams=1)
print("Random Forest classifier initialized.")

# Perform grid search
print("Performing grid search with cross-validation...")
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train_np, y_train_np)
print("Grid search completed.")

# Get the best model from grid search
print("Extracting the best model from grid search results...")
best_rf_clf = grid_search.best_estimator_
print("Best model extracted.")

# Evaluate the best model
print("Evaluating the best model on training data...")
train_accuracy = best_rf_clf.score(X_train_np, y_train_np)
print("Training Accuracy:", train_accuracy)

print("Evaluating the best model on testing data...")
test_accuracy = best_rf_clf.score(X_test_np, y_test_np)
print("Testing Accuracy:", test_accuracy)

# Obtain predictions on the test data using the best model
print("Obtaining predictions on the test data using the best model...")
rfc_test_predictions = best_rf_clf.predict(X_test_np)

# Print or use the test predictions
print("Test Predictions:", rfc_test_predictions)

# Save the best trained model for later use
print("Saving the best trained model to 'best_random_forest_model.pkl'...")
joblib.dump(best_rf_clf, 'best_random_forest_model.pkl')
print("Model saved successfully.")

Splitting data into training and testing sets...


NameError: name 'X' is not defined

In [16]:
np.unique(rfc_test_predictions, return_counts=True)

(array(['disappear', 'jumpingJacks', 'leftArm', 'leftFoot',
        'leftHandPinch', 'push', 'rest', 'tongue'], dtype='<U13'),
 array([2137, 2891, 7973, 5267, 2587, 1650, 4585, 2655]))

In [8]:
np.unique(y, return_counts=True)

(array([ 0,  2,  4,  6,  7,  8, 22, 34]),
 array([29087, 29077, 15428, 15186, 13927, 23532, 10531, 11957]))

In [1]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import cupy as cp
import joblib
import pickle

# Assuming X contains your 8x8 matrices and y contains the corresponding classes
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Encode y labels as numerical values
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert numpy arrays to cupy arrays for GPU acceleration
X_train_gpu = cp.array(X_train)
X_test_gpu = cp.array(X_test)
y_train_gpu = cp.array(y_train)
y_test_gpu = cp.array(y_test)

# Initializing and training the Gradient Boosting classifier
clf = GradientBoostingClassifier(n_estimators=1000, random_state=42)
clf.fit(cp.asnumpy(X_train_gpu), cp.asnumpy(y_train_gpu))

# Evaluate the model
train_accuracy = clf.score(cp.asnumpy(X_train_gpu), cp.asnumpy(y_train_gpu))
test_accuracy = clf.score(cp.asnumpy(X_test_gpu), cp.asnumpy(y_test_gpu))
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Obtain predictions on the test data
gboost_test_predictions = clf.predict(cp.asnumpy(X_test_gpu))

# Decode the predicted labels back to characters
gboost_test_predictions = label_encoder.inverse_transform(gboost_test_predictions)

# Print or use the test predictions
print("Test Predictions:", gboost_test_predictions)

# Save the trained model for later use
joblib.dump(clf, 'gradient_boosting_model.joblib')
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)
with open('standard_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


ModuleNotFoundError: No module named 'sklearn'

In [4]:
!ls -lart

total 2324041
-rw-rw-rw- 1 root root  204338555 May 19 07:02 output-balanced-classes-10000-or-more.json
-rw-rw-rw- 1 root root       1994 May 19 07:24 standard_scaler.pkl
-rw-rw-rw- 1 root root        655 May 19 07:24 label_encoder.pkl
-rw-rw-rw- 1 root root  112361717 May 19 07:33 random_forest_model.joblib
-rw-rw-rw- 1 root root 2059025686 May 19 07:48 best_random_forest_model.pkl
drwxrwxrwx 2 root root    1007480 May 19 09:14 .ipynb_checkpoints
-rw-rw-rw- 1 root root      11878 May 19 10:01 Untitled.ipynb
drwxr-xr-x 1 root root        124 May 19 14:10 ..
-rw-rw-rw- 1 root root      59111 May 19 15:05 train.ipynb
-rw-rw-rw- 1 root root       8024 May 19 15:06 inference.ipynb
drwxrwxrwx 3 root root    3000221 May 19 15:06 .


In [4]:
y

array([0, 5, 6, ..., 6, 6, 5])

In [5]:
np.unique(y, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([11957, 13927, 29077, 23532, 15428, 10531, 29087, 15186]))